In [1]:
import pandas as pd
import numpy as np
import json
import csv

In [11]:
scored_df = pd.read_csv("output/scores.csv")

subreddit_scores = {}

for idx, row in scored_df.iterrows():
    subreddit = row['community']
    partisan_score = row['partisan']

    subreddit_scores[subreddit] = partisan_score

subreddit_scores

{'keto': -0.0154967128061905,
 'AskReddit': 0.0528192829440357,
 'funny': 0.0626493272306779,
 'politics': -0.0714539587063294,
 'pics': 0.0578185310636667,
 'leagueoflegends': 0.1210424327392502,
 'gaming': 0.1043444004471168,
 'nfl': 0.0892897381442896,
 'worldnews': 0.0774050820557726,
 'videos': 0.1325866159823233,
 'nba': 0.0764697217082218,
 'WTF': 0.0743596906945835,
 'AdviceAnimals': 0.1164060589384081,
 'todayilearned': 0.0327101029695135,
 'news': 0.0409656747329308,
 'soccer': 0.0281439026480314,
 'The_Donald': 0.3351316374970577,
 'movies': 0.0202868500074103,
 'SquaredCircle': 0.0435756255017739,
 'hockey': 0.0388518605260938,
 'DotA2': 0.0974820880009292,
 'pcmasterrace': 0.1427182554349313,
 'CFB': 0.0732618423244495,
 'IAmA': 0.0409451068416209,
 'atheism': -0.0914974740741851,
 'trees': 0.0001875551433329,
 'DestinyTheGame': 0.0672181597949319,
 'GlobalOffensive': 0.1128348087179265,
 'gifs': 0.1103374932788082,
 'relationships': -0.0046412304229631,
 'aww': 0.00308844

In [35]:
with open('output/user_data_top100.json', 'r') as f:
    user_data = json.load(f)

len(user_data)

581

In [22]:


print(f"Top-level type: {type(user_data)} (should be dict)")

bad_users = []

for user, subreddits in user_data.items():
    if not isinstance(subreddits, dict):
        bad_users.append((user, type(subreddits)))

print(f"\nTotal users: {len(user_data)}")
print(f"Users with non-dict values: {len(bad_users)}")

if bad_users:
    print("\nExamples of broken entries:")
    for user, subr_type in bad_users[:5]:
        print(f"  {user}: {subr_type}")
else:
    print("✅ All users have dictionary-style subreddit data.")

Top-level type: <class 'dict'> (should be dict)

Total users: 581
Users with non-dict values: 143

Examples of broken entries:
  lotrouble: <class 'list'>
  dubalubdub: <class 'list'>
  Enjolras55: <class 'list'>
  Gnome_Sane: <class 'list'>
  hearsdemons: <class 'list'>


In [25]:
user_data['dubalubdub']

[]

In [26]:
    
user_partisan_scores = {}

for user, subreddits in user_data.items():
    if not isinstance(subreddits, dict) or not subreddits:
        # Skip empty or malformed entries
        user_partisan_scores[user] = None
        continue

    total_weight = 0
    weighted_sum = 0

    for subr, freq in subreddits.items():
        if subr in subreddit_scores:
            weighted_sum += freq * subreddit_scores[subr]
            total_weight += freq

    if total_weight > 0:
        user_partisan_scores[user] = weighted_sum / total_weight
    else:
        user_partisan_scores[user] = None

# === Output results ===
for user, score in user_partisan_scores.items():
    print(f"{user}: {score}")




Qu1nlan: -0.06296750452004397
english06: -0.02640135784827879
therealdanhill: 0.007498817620711929
Anxa: -0.06858114273806744
lotrouble: None
dottiemommy: -0.06548728356047431
Isentrope: -0.04361031914707362
captainmeta4: 0.04081112753717251
optimalg: -0.023820607318388056
Miskellaneousness: -0.07498809440898326
BagOnuts: -0.03832968129911796
wil_daven_: 0.01801741720776688
likeafox: -0.025111581996664394
MeghanAM: -0.03638034619441648
JonAce: -0.020283727741946193
The_Egalitarian: -0.05283370912773563
Precursor2552: -0.01114228491638352
Jakeable: 0.018500208531861857
PsychLegalMind: -0.006136709048676689
dubalubdub: None
pimanac: 0.35643742722248944
Enjolras55: None
nosecohn: 0.016879616125990586
amici_ursi: 0.027774787974495986
huadpe: 0.019750980989331663
travio: -0.023369521646499667
AcademicPublius: -0.0714539587063294
bulbasauuuur: -0.035886759902456926
ClockOfTheLongNow: -0.024703593147260394
premeddit: 0.03961946205952774
Infidel8: -0.040739421631867015
cassander: -0.0154353349

In [34]:
#pd.DataFrame.from_dict(user_partisan_scores, orient = 'columns')

user_df = pd.DataFrame(list(user_partisan_scores.items()), columns=["User", "Partisan_Score"])
user_df.to_csv("output/user_positioning.csv", index = False)